In [ ]:
import pandas as pd
import json
import os
from tqdm import tqdm

PARAMETERS = {
    'Solar8000/HR': 'Heart Rate',
    'Solar8000/ART_MBP': 'Arterial BP',
    'Solar8000/PLETH_SPO2': 'Oxygen Saturation',
    'Solar8000/ETCO2': 'End-Tidal CO2',
    'Solar8000/RR': 'Respiratory Rate',
    'Solar8000/PLETH_HR': 'Pleth HR',
    'Solar8000/ART_SBP': 'Systolic BP',
    'Solar8000/ART_DBP': 'Diastolic BP',
    'Solar8000/NIBP_MBP': 'NIBP Mean BP',
    'Solar8000/CVP': 'Central Venous Pressure',
    'Vigileo/CO': 'Cardiac Output',
    'Vigileo/SV': 'Stroke Volume',
    'Vigilance/HR_AVG': 'Average HR',
    'CardioQ/HR': 'CardioQ HR'
}

CASE_IDS = [25, 58, 68, 92, 96, 116, 135, 137, 142, 161, 203, 229, 239,
            243, 256, 266, 279, 323, 375, 384, 413, 416, 458, 460, 478]

TRKS_CSV_URL = 'https://api.vitaldb.net/trks'
TRACK_DATA_URL = 'https://api.vitaldb.net/{}.csv'


def interpolate_per_second(df):
    df['sec'] = df['time'].astype(int)
    df = df.drop_duplicates(subset='sec', keep='last')
    return df[['sec', 'value']].rename(columns={'sec': 'time'}).to_dict(orient='records')


print("📥 Downloading track metadata...")
tracks_df = pd.read_csv(TRKS_CSV_URL)
filtered_tracks = tracks_df[
    (tracks_df['caseid'].isin(CASE_IDS)) &
    (tracks_df['tname'].isin(PARAMETERS.keys()))
]

result = {}
for case_id in tqdm(CASE_IDS, desc="Cases"):
    case_tracks = filtered_tracks[filtered_tracks['caseid'] == case_id]
    if case_tracks.empty:
        print(f"[Case {case_id}] No matching parameters. Skipping.")
        continue

    case_data = {}
    for _, row in case_tracks.iterrows():
        label = PARAMETERS[row['tname']]
        tid = row['tid']
        try:
            df = pd.read_csv(TRACK_DATA_URL.format(tid), names=['time', 'value'], header=None)
            df.dropna(inplace=True)
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df.dropna(inplace=True)
            values = interpolate_per_second(df)
            case_data[label] = values
        except Exception as e:
            print(f"[Case {case_id} - {label}] Error loading tid {tid}: {e}")

    result[str(case_id)] = case_data

with open('vital_data.json', 'w') as f:
    json.dump(result, f, indent=2)

print("✅ vital_data.json saved.")





Cases:   4%|▍         | 1/25 [00:00<00:14,  1.61it/s]

[Case 25] HTTP error: 403


Cases:   8%|▊         | 2/25 [00:01<00:13,  1.70it/s]

[Case 58] HTTP error: 403


Cases:  12%|█▏        | 3/25 [00:01<00:13,  1.69it/s]

...

[Case 458] HTTP error: 403


Cases:  96%|█████████▌| 24/25 [00:14<00:00,  1.72it/s]

[Case 460] HTTP error: 403


Cases: 100%|██████████| 25/25 [00:14<00:00,  1.71it/s]

[Case 478] HTTP error: 403
✅ vital_data.json saved.
